In [1]:
import math
import numpy as np
from astropy.io.votable import parse_single_table
from astropy.coordinates import SkyCoord
from astropy.table import Column
import astropy.units as u
from astropy.io import ascii
import matplotlib.pyplot as plt

## Produce a latex table extract

### Source list

In [2]:
# Read the sources
parent_folder = ''
spec_table = parse_single_table(parent_folder+'gaskap_smc_spectra.vot').to_table()
spec_table

id,comp_name,ra,dec,rating,flux_peak,mean_cont,sd_cont,opacity_range,max_s_max_n,max_noise,num_chan_noise,min_opacity,vel_min_opacity,peak_tau,e_peak_tau,has_mw_abs,has_other_abs,semi_maj_axis,semi_min_axis,pa,nh_gass,noise_flag,beam_power,nh_uncorr,e_nh_uncorr_high,e_nh_uncorr_low,nh_corr,e_nh_corr_high,e_nh_corr_low,r_hi,e_r_hi_high,e_r_hi_low,f_int,mean_ts,e_mean_ts_high,e_mean_ts_low,ew,e_ew_high,e_ew_low,num_chan
,,deg,deg,,mJy/beam,mJy/beam,,,,,,,km / s,,,,,arcsec,arcsec,deg,1 / cm2,,,1 / cm2,1 / cm2,1 / cm2,1 / cm2,1 / cm2,1 / cm2,,,,,,,,km / s,km / s,km / s,
int64,str32,float64,float64,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,bool,float64,float64,float64,float64,bool,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1,J004725-712727,11.85596,-71.457707,B,15.015000343322754,0.0001577413873746991,0.20037295013116518,1.1125783920288086,1.0923846748301125,1.5317274332046509,43.0,0.41914910078048706,96.17224896304,0.869528591632843,0.3925101382105877,False,False,3.82400016784668,3.2720001220703128,133.6300048828125,6.174081421437779e+20,False,1.0359205,5.3681352738778035e+20,2.055848121587912e+19,2.0583805642083336e+19,6.102761578440182e+20,3.4751392622607598e+19,3.0792107892721582e+19,1.135551084172469,0.0407186152623733,0.03284365179935578,1.1028073395124844,49.98209873135783,15.397163964019093,10.247781470797385,7.717247360426716,2.2842825162824165,1.6082698574720897,40.0
5,J002620-743741,6.586833,-74.628167,C,15.201000213623047,0.00014202001038938762,0.26400873561229615,1.9746829271316528,1.4209718128364035,1.815657138824463,67.0,-0.15902580320835114,132.02632915333402,5.0,3.8316940181297285,False,False,4.056000137329102,3.3919998168945313,135.64999389648438,1.3315663738275294e+21,False,0.88999236,1.6599191705539466e+21,4.386579917446054e+19,4.068653288839604e+19,1.9359336432858002e+21,1.0774614638732352e+20,8.823405460989044e+19,1.1655428979441065,0.05136514084624855,0.042564961331798434,1.107899601157662,134.4544547327015,75.29091081534114,37.52897323244342,12.890402430359927,4.147494729692533,2.941461868429861,75.0
9,J011635-712604,19.148794,-71.434582,C,15.333000183105469,0.0001255630166269839,0.21174872823938362,1.6042497158050537,0.7604905305141251,1.9112515449523926,74.0,0.30700182914733887,185.81443163211898,1.1809015274047852,0.6544683758159434,False,False,3.8040000915527346,3.2080001831054688,142.02000427246094,1.227406986785527e+21,False,1.0161469,9.225906410409799e+20,3.1709622838580085e+19,2.8469665825475002e+19,1.0193064440086246e+21,5.644688378550891e+19,4.539505190344091e+19,1.1031294831257985,0.04499083352325273,0.03568619576330967,1.070964390989561,146.53632101005286,212.2463824431008,76.55952453457111,7.207530039675991,3.2779817575897567,2.514920429697532,85.0
10,J010425-742733,16.10635,-74.459193,C,15.435999870300293,0.00011435524793341756,0.2508074329566693,1.681606411933899,0.8255535251261913,1.9211487770080566,78.0,0.2395423799753189,75.45870833663803,1.4290249347686768,0.7164151873632499,False,False,3.7599998474121095,3.2759998321533206,139.1699981689453,3.131320668910356e+20,False,0.95265865,0.0,0.0,0.0,0.0,0.0,0.0,--,--,--,--,--,--,--,0.0,0.0,0.0,0.0
13,J005535-723533,13.899411,-72.592574,C,15.053000450134277,8.522021817043423e-05,0.16227310227980046,1.856298565864563,2.5452221979121297,1.5236057043075562,93.0,-0.33269286155700684,113.61597766529509,5.0,3.6192534660730047,False,True,5.552000045776367,4.431999969482422,140.7100067138672,7.446596934360939e+21,False,0.9928809,7.168226876960794e+21,6.6340713476586996e+19,6.726331614813487e+19,8.843626276630882e+21,3.628476748811773e+20,3.239942397242387e+20,1.2322084922320404,0.05165155267252364,0.041606787784398636,1.175805883533249,528.7622807422028,205.8532701106077,124.8570289946307,15.716832512486212,3.9184702066876436,2.7463186491828586,94.0
14,J012336-703232,20.902153,-70.542329,C,15.822999954

In [3]:
tau = []
ts = []
rhi = []
for row in spec_table:
    if row['min_opacity'] < 0:
        min_tau = row['peak_tau'] - row['e_peak_tau']
        if min_tau < 0:
            min_tau = 0
        tau_val = '$>{:.2f}$'.format(min_tau)
    else:
        tau_val = '${:.2f}\pm{:.2f}$'.format(row['peak_tau'], row['e_peak_tau'])
    tau.append(tau_val)
    ts_val = '${:.0f}^{{+{:.0f}}}_{{-{:.0f}}}$'.format(row['mean_ts'], row['e_mean_ts_high'], row['e_mean_ts_low'])
    ts.append(ts_val)
    rhi_val = '${:.1f}^{{+{:.2f}}}_{{-{:.2f}}}$'.format(row['r_hi'], row['e_r_hi_high'], row['e_r_hi_low'])
    rhi.append(rhi_val)

spec_table.add_column(Column(data=tau, name='tau'))
spec_table.add_column(Column(data=ts, name='mean_ts_disp'))
spec_table.add_column(Column(data=rhi, name='r_hi_disp'))

In [4]:
# Select a sample of sources surrounding the sample source
central_comp_name = 'J005141-725603' #'J005556-722605'
central_entry = spec_table[spec_table['comp_name']==central_comp_name]
central_entry
central_coord = SkyCoord(central_entry['ra'], central_entry['dec'])

all_coords = SkyCoord(spec_table['ra'], spec_table['dec'])
sample = all_coords.separation(central_coord) < 0.5*u.deg

In [5]:
sample_table = spec_table[sample]
sample_table.sort(['comp_name', 'ra', 'rating', 'id'])
sample_table

id,comp_name,ra,dec,rating,flux_peak,mean_cont,sd_cont,opacity_range,max_s_max_n,max_noise,num_chan_noise,min_opacity,vel_min_opacity,peak_tau,e_peak_tau,has_mw_abs,has_other_abs,semi_maj_axis,semi_min_axis,pa,nh_gass,noise_flag,beam_power,nh_uncorr,e_nh_uncorr_high,e_nh_uncorr_low,nh_corr,e_nh_corr_high,e_nh_corr_low,r_hi,e_r_hi_high,e_r_hi_low,f_int,mean_ts,e_mean_ts_high,e_mean_ts_low,ew,e_ew_high,e_ew_low,num_chan,tau,mean_ts_disp,r_hi_disp
,,deg,deg,,mJy/beam,mJy/beam,,,,,,,km / s,,,,,arcsec,arcsec,deg,1 / cm2,,,1 / cm2,1 / cm2,1 / cm2,1 / cm2,1 / cm2,1 / cm2,,,,,,,,km / s,km / s,km / s,,,,
int64,str32,float64,float64,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,bool,float64,float64,float64,float64,bool,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str13,str23,str21
178,J004718-723947,11.828467,-72.66331,B,32.32600021362305,0.0003151909913867712,0.13281936719594042,1.1794267892837524,1.1471056958771444,1.5493100881576538,62.0,0.369883269071579,143.75951202266705,0.9945678114891052,0.427704273049232,False,False,3.7919998168945312,3.279999923706055,137.39999389648438,4.4103578467198724e+21,False,1.0181546,3.961557153050681e+21,7.352576045918821e+19,7.617677407828207e+19,4.35511775332701e+21,1.0705171347257321e+20,1.1639028637096267e+20,1.0982903602641467,0.02178127331271673,0.020800663986599188,1.0746565793435676,265.3248119587797,75.7513284998397,44.98040844706327,10.556774755065952,2.5822289060742314,1.9709385697911126,94.0,$0.99\pm0.43$,$265^{+76}_{-45}$,$1.1^{+0.02}_{-0.02}$
304,J004956-723554,12.486259,-72.598535,A,81.70099639892578,0.0007445093709975481,0.033805358987482284,0.6342768669128418,4.180762669533237,1.1224292516708374,110.0,0.4881523549556732,141.6918558646755,0.7171277403831482,0.09901098194686853,True,True,4.6959999084472654,3.3520000457763675,129.13999938964844,4.815617270551048e+21,False,1.0075643,4.184251932658727e+21,8.295804215011154e+19,7.4434223685794e+19,4.521581020371164e+21,9.157663647450897e+19,8.670608032144294e+19,1.080076889431624,0.005078982450388292,0.004832368774581708,1.0785840379304847,320.8724317333969,25.676566043155788,20.973905289890467,8.735866101873901,0.5979310895286768,0.5493226573407991,102.0,$0.72\pm0.10$,$321^{+26}_{-21}$,$1.1^{+0.01}_{-0.00}$
33,J005014-730326,12.561714,-73.057288,C,16.516000747680664,0.00013204875867813825,0.19689896771242069,1.833931803703308,1.7889600056779422,1.657568335533142,83.0,-0.17636345326900482,156.91375551519,5.0,3.878704994296605,False,True,4.231999969482422,3.7040000915527345,124.37999725341797,9.285400120443429e+21,False,0.98851216,1.0307584032510973e+22,8.14803393800768e+19,7.749701068357265e+19,1.2700597355956283e+22,4.933249604324174e+20,4.573327253481162e+20,1.2331764335373825,0.04435337172582843,0.0450013346247049,1.157342694331896,481.75022941069574,146.68927904157601,97.29383142071788,23.399996845350465,5.104531576081085,3.5029318221153996,105.0,$>1.12$,$482^{+147}_{-97}$,$1.2^{+0.04}_{-0.05}$
74,J005141-725545,12.9231,-72.929259,B,17.329999923706055,0.00013370290398597717,0.13590231534036942,1.4701125621795654,2.3030261453337606,1.4450805187225342,32.0,-0.02503207139670849,117.83179317180102,5.0,3.5529573115458684,False,True,4.536000061035156,4.22400016784668,166.7899932861328,8.128669527708004e+21,False,0.98470074,8.611043046609593e+21,8.036740590818925e+19,8.755433561629865e+19,1.1299564685474224e+22,4.6352963954871475e+20,3.640237808755933e+20,1.3117023388170908,0.0520534629718048,0.03949812231444261,1.25256410844642,259.611767998971,32.05352363922606,26.142892768986684,28.44735438462521,4.122393569136584,2.819660803794362,103.0,$>1.45$,$260^{+32}_{-26}$,$1.3^{+0.05}_{-0.04}$
75,J005141-725603,12.921625,-72.934274,B,19.326000213623047,0.000139997573569417,0.13743164716621264,1.586621642112732,3.0488391892919613,1.3918707370758057,53.0,-0.1947508603334427,117.8343473909923,5.0,3.56419

In [6]:
# Select the columns to be output
sample_table.keep_columns(['id', 'comp_name', 'ra', 'dec', 'flux_peak', 'rating', 'sd_cont', 'tau', 'nh_uncorr', 'mean_ts_disp', 'r_hi_disp'])
sample_table = sample_table['id', 'comp_name', 'ra', 'dec', 'flux_peak', 'rating', 'sd_cont', 'tau', 'nh_uncorr', 'mean_ts_disp', 'r_hi_disp']

In [7]:
# Set up formatting of the output table

# Column titles
names = ['ID', 'Source', 'RA', 'Dec', 'Peak Flux', 'Rating', r'$\sigma_{\rm cont}$', r'Peak $\tau$', r'N$_{\rm HI,uncorr}$', r'$\langle T_{\rm S} \rangle$', r'$\mathcal{R}_{\rm HI}$']

# Note the names need to be the ones in the table output, as defined above
formats={'Source': lambda x: x.replace('-', '$-$'), 'RA': '%.4f', 'Dec': '$%.3f$', 'Peak Flux' : '%.0f', names[6] : '%.3f', 
         names[-3] : lambda x: str(round(x/1e21, 1))}

# We have to set the units manually to get the brackets around the units
units = {'RA':'(deg)', 'Dec':'(deg)', 'Peak Flux': '(mJy)', names[-3] : r'($10^{21}$ cm$^{-2}$)'}

col_align=''.join(['c' if name in ('Source', 'Rating') else 'r'  for name in names])
latextdict = {'units': units, 'tabletype': None, 'header_start': '\\hline \\hline', 
              'header_end': '\\hline', 'data_end': '\\hline'}

In [8]:
sample_table

id,comp_name,ra,dec,flux_peak,rating,sd_cont,tau,nh_uncorr,mean_ts_disp,r_hi_disp
,,deg,deg,mJy/beam,,,,1 / cm2,,
int64,str32,float64,float64,float64,str1,float64,str13,float64,str23,str21
178,J004718-723947,11.828467,-72.66331,32.32600021362305,B,0.13281936719594042,$0.99\pm0.43$,3.961557153050681e+21,$265^{+76}_{-45}$,$1.1^{+0.02}_{-0.02}$
304,J004956-723554,12.486259,-72.598535,81.70099639892578,A,0.033805358987482284,$0.72\pm0.10$,4.184251932658727e+21,$321^{+26}_{-21}$,$1.1^{+0.01}_{-0.00}$
33,J005014-730326,12.561714,-73.057288,16.516000747680664,C,0.19689896771242069,$>1.12$,1.0307584032510973e+22,$482^{+147}_{-97}$,$1.2^{+0.04}_{-0.05}$
74,J005141-725545,12.9231,-72.929259,17.329999923706055,B,0.13590231534036942,$>1.45$,8.611043046609593e+21,$260^{+32}_{-26}$,$1.3^{+0.05}_{-0.04}$
75,J005141-725603,12.921625,-72.934274,19.326000213623047,B,0.13743164716621264,$>1.44$,8.620509761524245e+21,$251^{+35}_{-24}$,$1.3^{+0.05}_{-0.04}$
62,J005217-730157,13.071374,-73.032638,17.68000030517578,B,0.1457856542944897,$1.28\pm0.63$,7.813050829887583e+21,$291^{+47}_{-39}$,$1.2^{+0.04}_{-0.04}$
357,J005218-722708,13.077894,-72.4523,153.1790008544922,B,0.018857461197800653,$0.22\pm0.04$,3.8956810830235147e+21,$937^{+131}_{-94}$,$1.0^{+0.00}_{-0.00}$
319,J005238-731245,13.158857,-73.212613,100.67900085449219,A,0.03533799362513302,$3.26\pm0.92$,8.903994856330093e+21,$234^{+7}_{-6}$,$1.4^{+0.03}_{-0.02}$


In [9]:

ascii.write(sample_table, 'spectra_table.tex', Writer=ascii.Latex, latexdict=latextdict, 
            formats=formats, names=names, col_align=col_align, overwrite=True)

## Absorption List

In [10]:
#parent_folder = 'sb10944/v3a/'
parent_folder = ''
abs_table = parse_single_table(parent_folder+'gaskap_smc_absorption.vot').to_table()
abs_table



id,comp_name,abs_name,ra,dec,rating,flux_peak,mean_cont,sd_cont,opacity_range,max_s_max_n,max_noise,num_chan_noise,semi_maj_axis,semi_min_axis,pa,start_vel,end_vel,length,min_optical_depth,e_min_optical_depth,peak_tau,e_peak_tau,max_sigma
,,,deg,deg,,mJy/beam,mJy/beam,,,,,,arcsec,arcsec,deg,km / s,km / s,chan,,,,,
int64,str32,str32,float64,float64,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64
13,J005535-723533,J005535-723533_113,13.899411,-72.592574,C,15.053000450134277,8.522021817043423e-05,0.16227310227980046,1.856298565864563,2.5452221979121297,1.5236057043075562,93.0,5.552000045776367,4.431999969482422,140.7100067138672,113.61597766529509,114.6159776652694,2,-0.33269286155700684,0.2513908112211382,5.0,3.6192534660730047,5.30127913221415
13,J005535-723533,J005535-723533_116,13.899411,-72.592574,C,15.053000450134277,8.522021817043423e-05,0.16227310227980046,1.856298565864563,2.5452221979121297,1.5236057043075562,93.0,5.552000045776367,4.431999969482422,140.7100067138672,116.615977665218,117.6159776651923,2,-0.06738349795341492,0.27018215923557837,5.0,3.6913411163683785,3.9506069933546564
19,J005448-725353,J005448-725353_116,13.702572,-72.898137,C,15.98799991607666,0.00015485620824620127,0.18070838215007914,2.211228370666504,2.0076773559084584,1.7351946830749512,52.0,3.8840000152587892,3.2680000305175785,137.1300048828125,116.77713590216,118.77713590224198,3,-0.12201661616563797,0.310779098902246,5.0,3.8313270879007435,3.6103350330925146
19,J005448-725353,J005448-725353_163,13.702572,-72.898137,C,15.98799991607666,0.00015485620824620127,0.18070838215007914,2.211228370666504,2.0076773559084584,1.7351946830749512,52.0,3.8840000152587892,3.2680000305175785,137.1300048828125,163.77713590408698,167.77713590425103,5,-0.4760337173938751,0.3077864799197406,5.0,3.821651016570783,4.795641731815016
26,J010532-721331,J010532-721331_131,16.386939,-72.225495,C,16.305999755859375,0.00013356159906834363,0.20625884450647666,1.7323534488677979,1.4747387200154491,1.7000147104263306,105.0,3.7759998321533206,3.2279998779296877,138.6999969482422,131.318490585975,133.318490586057,3,-0.03233879804611206,0.24499679862438659,5.0,3.59348986463798,4.2136830499302675
33,J005014-730326,J005014-730326_152,12.561714,-73.057288,C,16.516000747680664,0.00013204875867813825,0.19689896771242069,1.833931803703308,1.7889600056779422,1.657568335533142,83.0,4.231999969482422,3.7040000915527345,124.37999725341797,152.9137555152928,153.9137555152671,2,-0.06129693612456322,0.32034791657862954,5.0,3.8616523655032515,3.3129509665138017
42,J013134-700042,J013134-700042_240,22.893692,-70.011737,B,15.736000061035156,9.92985675111413e-05,0.10793941277669811,1.12614905834198,0.8512510925074565,1.6083178520202637,139.0,7.744000244140626,3.9200000762939453,120.55000305175781,240.989911641826,241.989911641867,2,0.6641514301300049,0.10801005662009627,0.409245103597641,0.15068353249336164,3.1094194409255347
60,J012924-733153,J012924-733153_179,22.350415,-73.531439,B,18.4689998626709,0.00013900268822908401,0.21099784313347633,1.3799264430999756,1.4701763786530915,1.5586347579956055,53.0,3.808000183105469,3.2680000305175785,142.8000030517578,179.78932071129887,180.78932071127315,2,0.2738746702671051,0.23944895698410754,1.2950847148895264,0.6282359345160137,3.032484954960196


In [11]:
tau = []
od = []
for row in abs_table:
    if row['min_optical_depth'] < 0:
        min_tau = row['peak_tau'] - row['e_peak_tau']
        if min_tau < 0:
            min_tau = 0
        tau_val = '$>{:.2f}$'.format(min_tau)
    else:
        tau_val = '${:.2f}\pm{:.2f}$'.format(row['peak_tau'], row['e_peak_tau'])
    tau.append(tau_val)
    od_val = '${:.2f}\pm{:.2f}$'.format(row['min_optical_depth'], row['e_min_optical_depth'])
    od.append(od_val)
abs_table.add_column(Column(data=tau, name='tau'))
abs_table.add_column(Column(data=od, name='optical_depth'))

In [12]:
abs_sample = np.isin(abs_table['id'], spec_table[sample]['id'])
abs_sample_table = abs_table[abs_sample]

In [13]:
abs_sample_table.sort(['comp_name', 'abs_name'])

# Select the columns to be output
abs_sample_table.keep_columns(['comp_name', 'abs_name', 'start_vel', 'end_vel', 'length', 'optical_depth', 'tau', 'max_sigma'])
abs_sample_table = abs_sample_table['comp_name', 'abs_name', 'start_vel', 'end_vel', 'length', 'optical_depth', 'tau', 'max_sigma']
abs_sample_table

comp_name,abs_name,start_vel,end_vel,length,optical_depth,tau,max_sigma
,,km / s,km / s,chan,,,
str32,str32,float64,float64,int64,str14,str13,float64
J004956-723554,J004956-723554_-2,-2.3081441316237323,-0.3081441316751298,3,$0.84\pm0.04$,$0.18\pm0.05$,4.056181693914019
J004956-723554,J004956-723554_137,137.6918558647783,150.69185586444416,14,$0.49\pm0.05$,$0.72\pm0.10$,10.074527449013305
J004956-723554,J004956-723554_155,155.69185586431564,160.69185586418718,6,$0.55\pm0.05$,$0.60\pm0.09$,8.70680109815613
J004956-723554,J004956-723554_3,3.6918558682220755,4.6918558681963765,2,$0.82\pm0.04$,$0.19\pm0.05$,4.6257749581907825
J005014-730326,J005014-730326_152,152.9137555152928,153.9137555152671,2,$-0.06\pm0.32$,$>1.14$,3.3129509665138017
J005141-725545,J005141-725545_106,106.83179317135003,108.83179317143201,3,$0.29\pm0.19$,$1.24\pm0.51$,3.7532282753283317
J005141-725545,J005141-725545_114,114.83179317167803,117.83179317180102,4,$-0.03\pm0.24$,$>1.45$,4.531221915961278
J005141-725603,J005141-725603_113,113.8343473910951,120.83434739091521,8,$-0.19\pm0.24$,$>1.44$,5.0215491084522


In [14]:
# Set up formatting of the output table

# Column titles
names = ['Source', 'Feature', 'Min Velocity', 'Max Velocity', 'Width', 'Min Optical', r'Peak $\tau$', 'Significance']

# Note the names need to be the ones in the table output, as defined above
formats={'Source': lambda x: x.replace('-', '$-$'), 'Feature': lambda x: x.replace('-', '$-$').replace('_', '\_'), 
         'Min Velocity': '$%.1f$', 'Max Velocity': '$%.1f$', 'Significance' : '%.1f'}

# We have to set the units manually to get the brackets around the units
units = {'Min Velocity':r'(kms$^{-1}$)', 'Max Velocity':r'(kms$^{-1}$)', 'Width':r'(kms$^{-1}$)', 
         'Significance': '(sigma)', 'Min Optical':'Depth'}

col_align=''.join(['l' if name in ('Source', 'Feature') else 'r'  for name in names])
latextdict = {'units': units, 'tabletype': None, 'header_start': '\\hline \\hline', 
              'header_end': '\\hline', 'data_end': '\\hline'}

In [15]:

ascii.write(abs_sample_table, 'absorption_table.tex', Writer=ascii.Latex, latexdict=latextdict, 
            formats=formats, names=names, col_align=col_align, overwrite=True)


### Detections stats

In [16]:
spec_table

id,comp_name,ra,dec,rating,flux_peak,mean_cont,sd_cont,opacity_range,max_s_max_n,max_noise,num_chan_noise,min_opacity,vel_min_opacity,peak_tau,e_peak_tau,has_mw_abs,has_other_abs,semi_maj_axis,semi_min_axis,pa,nh_gass,noise_flag,beam_power,nh_uncorr,e_nh_uncorr_high,e_nh_uncorr_low,nh_corr,e_nh_corr_high,e_nh_corr_low,r_hi,e_r_hi_high,e_r_hi_low,f_int,mean_ts,e_mean_ts_high,e_mean_ts_low,ew,e_ew_high,e_ew_low,num_chan,tau
,,deg,deg,,mJy/beam,mJy/beam,,,,,,,km / s,,,,,arcsec,arcsec,deg,1 / cm2,,,1 / cm2,1 / cm2,1 / cm2,1 / cm2,1 / cm2,1 / cm2,,,,,,,,km / s,km / s,km / s,,
int64,str32,float64,float64,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,bool,float64,float64,float64,float64,bool,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str13
1,J004725-712727,11.85596,-71.457707,B,15.015000343322754,0.0001577413873746991,0.20037295013116518,1.1125783920288086,1.0923846748301125,1.5317274332046509,43.0,0.41914910078048706,96.17224896304,0.869528591632843,0.3925101382105877,False,False,3.82400016784668,3.2720001220703128,133.6300048828125,6.174081421437779e+20,False,1.0359205,5.3609459556020525e+20,2.005383840086845e+19,1.939808064044309e+19,6.080017259252707e+20,3.2786030026031497e+19,2.9008250756933026e+19,1.1332013732386819,0.03864555841538997,0.03178974766492071,1.101661962410547,49.594115043646525,17.923253593319828,9.726427506354817,7.610656601077766,2.2798002187408777,1.6022098980791175,40.0,$0.87\pm0.39$
5,J002620-743741,6.586833,-74.628167,C,15.201000213623047,0.00014202001038938762,0.26400873561229615,1.9746829271316528,1.4209718128364035,1.815657138824463,67.0,-0.15902580320835114,132.02632915333402,5.0,3.8316940181297285,False,False,4.056000137329102,3.3919998168945313,135.64999389648438,1.3315663738275294e+21,False,0.88999236,1.6617698350880721e+21,4.473961905960832e+19,4.156131897077504e+19,1.9324069311289827e+21,1.0513999383527817e+20,8.76760108291402e+19,1.1613661870603131,0.06044848101918654,0.043170264384253754,1.105132618689647,139.05472105263345,90.66319719868918,42.611091722799316,12.661276057686171,4.0930303956642495,2.9246159621139336,75.0,$>1.17$
9,J011635-712604,19.148794,-71.434582,C,15.333000183105469,0.0001255630166269839,0.21174872823938362,1.6042497158050537,0.7604905305141251,1.9112515449523926,74.0,0.30700182914733887,185.81443163211898,1.1809015274047852,0.6544683758159434,False,False,3.8040000915527346,3.2080001831054688,142.02000427246094,1.227406986785527e+21,False,1.0161469,9.2425442732153e+20,3.1546231610364264e+19,2.8407582935803625e+19,1.0247457152677377e+21,5.496362188035536e+19,5.161475710483038e+19,1.1032102679835034,0.047230896491608165,0.035454740205925095,1.0690745722945996,152.47350781706277,186.9962447509831,76.4069557014094,7.087259787690703,3.336071329518139,2.455134863999242,85.0,$1.18\pm0.65$
10,J010425-742733,16.10635,-74.459193,C,15.435999870300293,0.00011435524793341756,0.2508074329566693,1.681606411933899,0.8255535251261913,1.9211487770080566,78.0,0.2395423799753189,75.45870833663803,1.4290249347686768,0.7164151873632499,False,False,3.7599998474121095,3.2759998321533206,139.1699981689453,3.131320668910356e+20,False,0.95265865,0.0,0.0,0.0,0.0,0.0,0.0,--,--,--,--,--,--,--,0.0,0.0,0.0,0.0,$1.43\pm0.72$
13,J005535-723533,13.899411,-72.592574,C,15.053000450134277,8.522021817043423e-05,0.16227310227980046,1.856298565864563,2.5452221979121297,1.5236057043075562,93.0,-0.33269286155700684,113.61597766529509,5.0,3.6192534660730047,False,True,5.552000045776367,4.431999969482422,140.7100067138672,7.446596934360939e+21,False,0.9928809,7.168789096883223e+21,6.487623649317932e+19,7.132487846458294e+19,8.836048422052917e+21,3.705861370433084e+20,3.2768374001223755e+20,1.2308184612966393,0.05013477581983139,0.04086736181039963,1.1767830163216406,512.831152138348,249.03820563326394,120.6341644117548,15.77317655134849,3.850867842911161,2.72838467838472

In [17]:
rows= []
num_recs = len(spec_table)
num_det = np.sum(spec_table['has_other_abs'])
rate = num_det / num_recs * 100
print ("Found {} detections from {} sources ({:.0f} %)".format(num_det, num_recs, rate))
non_dets = spec_table[~spec_table['has_other_abs']]
max_non_det = np.max(non_dets['flux_peak'])
print ("Non detections - max peak flux {:.2f} lowest noise {:.3f}".format(max_non_det, np.min(non_dets['sd_cont'])))
#print ("{} & {} & {:.0f} \\".format(num_recs, num_det, rate))
rows.append(("All",num_recs, num_det, rate))

Found 65 detections from 229 sources (28 %)
Non detections - max peak flux 423.12 lowest noise 0.009


In [18]:
smc_sample = spec_table[spec_table['nh_uncorr'] >= 2e21]
num_smc_recs = len(smc_sample)
num_smc_det = np.sum(smc_sample['has_other_abs'])
smc_rate = num_smc_det / num_smc_recs * 100
print ("Found {} detections from {} SMC sources ({:.0f} %)".format(num_smc_det, num_smc_recs, smc_rate))

non_dets = smc_sample[~smc_sample['has_other_abs']]
max_non_det = np.max(non_dets['flux_peak'])
print ("Non detections - max peak flux {:.2f} lowest noise {:.3f}".format(max_non_det, np.min(non_dets['sd_cont'])))

#print ("{} & {} & {:.0f} \\".format(num_smc_recs, num_smc_det, smc_rate))
rows.append(("SMC Body",num_smc_recs, num_smc_det, smc_rate))

Found 49 detections from 79 SMC sources (62 %)
Non detections - max peak flux 38.45 lowest noise 0.073


In [19]:
non_smc_sample = spec_table[spec_table['nh_uncorr'] < 2e21]
num_non_smc_recs = len(non_smc_sample)
num_non_smc_det = np.sum(non_smc_sample['has_other_abs'])
non_smc_rate = num_non_smc_det / num_non_smc_recs * 100
print ("Found {} detections from {} non-SMC sources ({:.0f} %)".format(num_non_smc_det, num_non_smc_recs, non_smc_rate))

non_dets = non_smc_sample[~non_smc_sample['has_other_abs']]
max_non_det = np.max(non_dets['flux_peak'])
print ("Non detections - max peak flux {:.2f} lowest noise {:.3f}".format(max_non_det, np.min(non_dets['sd_cont'])))

#print ("{} & {} & {:.0f} \\".format(num_non_smc_recs, num_non_smc_det, non_smc_rate))
rows.append(("Outside SMC",num_non_smc_recs, num_non_smc_det, non_smc_rate))

Found 16 detections from 150 non-SMC sources (11 %)
Non detections - max peak flux 423.12 lowest noise 0.009


In [20]:
for row in rows:
    print ("{} & {} & {} & {:.0f} \\\\".format(row[0], row[1], row[2], row[3]))

All & 229 & 65 & 28 \\
SMC Body & 79 & 49 & 62 \\
Outside SMC & 150 & 16 & 11 \\
